In [1]:
# Make sure you have a GPU running
!nvidia-smi

Tue Dec  1 04:46:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html


  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-c3j6xq68
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-c3j6xq68
     |████████████████████████████████| 215kB 8.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 327kB 13.3MB/s 
     |████████████████████████████████| 327kB 16.7MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
     |████████████████████████████████| 983kB 19.4MB/s 
     |████████████████████████████████| 5.5MB 22.5MB/s 
     |████████████████████████████████| 143kB 31.9MB/s 
     |████████████████████████████████| 225kB 36.4MB/s 
     |████████████████████████████████| 3.0MB 31.4MB/s 
     |████████████████████████████████| 7.2MB 49.3MB/s 
     |████████████████████████████████| 3.9MB 27.0MB/s 
     |████████████████████████████████

In [3]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

12/01/2020 04:52:04 - INFO - faiss -   Loading faiss with AVX2 support.
12/01/2020 04:52:04 - INFO - faiss -   Loading faiss.


In [4]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()

In [5]:
# Let's first get some files that we want to use
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

12/01/2020 04:53:45 - INFO - haystack.preprocessor.utils -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1095120/1095120 [00:00<00:00, 1292963.62B/s]
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/304_Hardhome.txt
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/121_The_Bear_and_the_Maiden_Fair.txt
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/408_The_Last_of_the_Starks.txt
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/443_The_Kingsroad.txt
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/191_Gendry.txt
12/01/2020 04:53:47 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/262_Gilly__character_.txt
12/01/2020 04:53:47 - INFO -

In [7]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

12/01/2020 04:56:46 - INFO - filelock -   Lock 140256233354296 acquired on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


12/01/2020 04:56:46 - INFO - filelock -   Lock 140256233354296 released on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


12/01/2020 04:56:47 - INFO - filelock -   Lock 140256222131536 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


12/01/2020 04:56:47 - INFO - filelock -   Lock 140256222131536 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


12/01/2020 04:56:48 - INFO - filelock -   Lock 140256189501112 acquired on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


...

12/01/2020 04:57:06 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/01/2020 04:57:06 - INFO - filelock -   Lock 140256203319728 acquired on /root/.cache/torch/transformers/f6388f32b32eac5dad8f0f9c7009ce69e967c1b65ebae62f805fced8022ea991.9500f04f28d7c0ca5f9c265db7ba5030897a2d752451412827f7dec185b1ee36.lock


12/01/2020 04:57:07 - INFO - filelock -   Lock 140256203319728 released on /root/.cache/torch/transformers/f6388f32b32eac5dad8f0f9c7009ce69e967c1b65ebae62f805fced8022ea991.9500f04f28d7c0ca5f9c265db7ba5030897a2d752451412827f7dec185b1ee36.lock


12/01/2020 04:57:08 - INFO - filelock -   Lock 140256194574600 acquired on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


12/01/2020 04:57:14 - INFO - filelock -   Lock 140256194574600 released on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


12/01/2020 04:57:19 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/01/2020 04:57:27 - INFO - haystack.document_store.faiss -   Updating embeddings for 2497 docs...
Inferencing Samples: 100%|██████████| 157/157 [00:34<00:00,  4.49 Batches/s]
12/01/2020 04:58:05 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00,  3.53it/s]


In [8]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/01/2020 05:10:41 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
12/01/2020 05:10:41 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
12/01/2020 05:10:41 - INFO - filelock -   Lock 140256188172048 acquired on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.6060f348ba2b58d6d30b5324910152ffc512e7c3891ed13f22844f1a9b5c0d0f.lock


12/01/2020 05:10:41 - INFO - filelock -   Lock 140256188172048 released on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.6060f348ba2b58d6d30b5324910152ffc512e7c3891ed13f22844f1a9b5c0d0f.lock


12/01/2020 05:10:42 - INFO - filelock -   Lock 140256185446016 acquired on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.a851909c96149f062acca04d647da88d0dcd3a52cd5a8c7169e89fc6e5971c7b.lock


12/01/2020 05:10:57 - INFO - filelock -   Lock 140256185446016 released on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.a851909c96149f062acca04d647da88d0dcd3a52cd5a8c7169e89fc6e5971c7b.lock


Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/01/2020 05:11:00 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/01/2020 05:11:06 - INFO - filelock -   Lock 140256191465232 acquired on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


...

12/01/2020 05:11:08 - INFO - filelock -   Lock 140256191586600 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.6e217123a3ada61145de1f20b1443a1ec9aac93492a4bd1ce6a695935f0fd97a.lock


12/01/2020 05:11:08 - INFO - filelock -   Lock 140256191586600 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.6e217123a3ada61145de1f20b1443a1ec9aac93492a4bd1ce6a695935f0fd97a.lock


12/01/2020 05:11:09 - INFO - filelock -   Lock 140256191466408 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.ec06af3e1b426682955dab3bd553eaf178b6eafac9079fc133925e0e2654213e.lock


12/01/2020 05:11:09 - INFO - filelock -   Lock 140256191466408 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.ec06af3e1b426682955dab3bd553eaf178b6eafac9079fc133925e0e2654213e.lock


12/01/2020 05:11:10 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
12/01/2020 05:11:10 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
12/01/2020 05:11:10 - INFO - farm.infer -    0 
12/01/2020 05:11:10 - INFO - farm.infer -   /w\
12/01/2020 05:11:10 - INFO - farm.infer -   /'\
12/01/2020 05:11:10 - INFO - farm.infer -   


In [9]:
finder = Finder(reader, retriever)

12/01/2020 05:13:05 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [14]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_retriever=10, top_k_reader=5)

prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_retriever=10, top_k_reader=5)

12/01/2020 05:17:42 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 57.16 Batches/s]
12/01/2020 05:17:42 - INFO - haystack.finder -   Got 10 candidates from retriever
12/01/2020 05:17:42 - INFO - haystack.finder -   Reader is looking for detailed answer in 10951 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 16.86 Batches/s]


In [15]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eddard and Catelyn Stark',
        'context': 'Background ===\n'
                   'Arya is the third child and younger daughter of Eddard and '
                   'Catelyn Stark and is nine years old at the beginning of '
                   'the book series.  Sh'},
    {   'answer': 'Eddard "Ned" Stark',
        'context': 'haracter description ==\n'
                   'Rickon is the fifth and youngest child of Eddard "Ned" '
                   'Stark and his wife Catelyn, and has five siblings—Robb, '
                   'Sansa, Arya, Br'},
    {   'answer': 'Eddard Stark',
        'context': '\n'
                   '=== Eddard Stark ===\n'
                   'Eddard "Ned" Stark is the Lord of Winterfell and Warden of '
                   'the North, and briefly serves as Hand of the King to '
                   'Robert Baratheon'},
    {   'answer': 'Rhaegar',
        'context': ', Aemon Targaryen, Jorah Mormont, Meera Reed, Jon '
                   'Connington an